In [ ]:
# STEP 0: Install dependencies
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 langgraph langchain langchain-google-genai openai matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.7/767.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.5 MB/s eta 0:00:00


In [ ]:
# STEP 1: Imports and secure API key input
import os
import matplotlib.pyplot as plt
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import re

# Secure Gemini API Key input (for Colab use only)
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
# STEP 2: Initialize Gemini
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0.2, google_api_key=GOOGLE_API_KEY)

In [ ]:
# STEP 3: Ask user for basic details before goal

def get_user_input(state: dict) -> dict:
    print("Namaste 🙏! I am NutriAI, your Indian dietician. Let's begin your health journey!")
    name = input("👤 What's your name? ")
    age = int(input("🎂 Your age: "))
    gender = input("⚧️ Your gender (male/female/other): ").strip().lower()
    goal = input("🏁 Your health goal? (weight loss/gain, healthy diet): ")
    preference = input("🌿 Diet preference (veg/non-veg/vegan): ")
    region = input("🍛 Preferred Indian cuisine region: ")
    health_issue = input("🩺 Any health issues? (PCOS, acne, thyroid, etc.): ")
    height = float(input("📏 Height in cm: ").strip())
    weight = float(input("⚖️ Weight in kg: ").strip())
    activity_level = input("🏃 Activity level (low, moderate, high): ").strip().lower()

    state.update({"name": name, "age": age, "gender": gender, "goal": goal, "preference": preference, "region": region, "health_issue": health_issue, "height": height, "weight": weight, "activity_level": activity_level})
    return state

# STEP 4: Classify the user's dietary category

def classify_goal(state: dict) -> dict:
    goal = state['goal']
    if "loss" in goal.lower():
        category = "Weight Loss"
    elif "gain" in goal.lower():
        category = "Weight Gain"
    else:
        category = "Balanced Diet"
    print(f"🎯 Goal classified as: {category}")
    state["category"] = category
    return state

# STEP 5: Route based on goal

def diet_router(state: dict) -> str:
    cat = state["category"].lower()
    if "loss" in cat:
        return "loss"
    elif "gain" in cat:
        return "gain"
    return "balanced"

In [ ]:
# STEP 6: Generate personalized meal plan

def generate_plan(state: dict, goal_type: str) -> dict:
    soft = "Prefer soft/easy-to-digest foods." if state["age"] > 50 else ""
    prompt = f"Namaste {state['name']} 🙏! I am NutriAI, your Indian dietician. Let's create your Indian meal plan. Goal: {state['goal']} ({goal_type}) Age: {state['age']}, Gender: {state['gender']}, Region: {state['region']}, Diet: {state['preference']}, Health Issues: {state['health_issue']}. {soft} Suggest a full day's meal plan with traditional foods."
    response = llm.invoke([HumanMessage(content=prompt)])
    state["plan"] = response.content.strip()
    print("\n🍽️ Your Personalized Diet Plan:\n")
    print(state["plan"])
    return state

def loss_node(state): return generate_plan(state, "Weight Loss")
def gain_node(state): return generate_plan(state, "Weight Gain")
def balanced_node(state): return generate_plan(state, "Balanced")

# STEP 7: Nutrition calculator

def nutrition_summary(state: dict) -> dict:
    bmr = 10 * state['weight'] + 6.25 * state['height'] - 5 * state['age']
    bmr += -161 if state['gender'] == 'female' else 5
    factor = {'low': 1.2, 'moderate': 1.55, 'high': 1.725}[state['activity_level']]
    calories = bmr * factor
    protein = state['weight'] * 1.2
    state.update({"calories": round(calories), "protein": round(protein)})
    print("\n📊 Daily Nutrition Goal:")
    print(f"Calories: {state['calories']} kcal, Protein: {state['protein']} g")
    return state

In [ ]:
# STEP 8: Track what user ate

def track_meals(state: dict) -> dict:
    print("\n🧾 What did you eat today:")
    meals = {meal: input(f"🍽️ {meal.capitalize()}: ") for meal in ['breakfast', 'lunch', 'dinner', 'snacks']}
    meal_summary = "\n".join([f"{m.capitalize()}: {v}" for m,v in meals.items()])
    prompt = f"Estimate total calories and protein for:\n{meal_summary}\nFormat: Calories: X kcal, Protein: Y g"
    response = llm.invoke([HumanMessage(content=prompt)])
    state.update({"meal_summary": meal_summary, "meal_analysis": response.content.strip()})
    print("\n📋 Logged Intake:")
    print(response.content.strip())
    return state

# STEP 9: Remaining calories/protein

def track_remaining(state: dict) -> dict:
    cal = re.search(r'Calories\D+(\d+)', state["meal_analysis"])
    prot = re.search(r'Protein\D+(\d+)', state["meal_analysis"])
    cals = int(cal.group(1)) if cal else 0
    prots = int(prot.group(1)) if prot else 0
    rc, rp = max(state['calories'] - cals, 0), max(state['protein'] - prots, 0)
    print("\n📉 Remaining Today:")
    print(f"Calories left: {rc} kcal, Protein left: {rp} g")
    return state

# STEP 10: Recommend exercise

def suggest_exercise(state: dict) -> dict:
    prompt = f"Suggest a short Indian home workout (can include yoga) for Goal: {state['goal']}, Gender: {state['gender']}, Age: {state['age']}, Health: {state['health_issue']}"
    response = llm.invoke([HumanMessage(content=prompt)])
    print("\n🏃 Exercise/Yoga:")
    print(response.content.strip())
    state["exercise"] = response.content.strip()
    return state

In [ ]:
# STEP 11: LangGraph flow

builder = StateGraph(dict)
builder.set_entry_point("get_user_input")
builder.add_node("get_user_input", get_user_input)
builder.add_node("classify", classify_goal)
builder.add_node("loss", loss_node)
builder.add_node("gain", gain_node)
builder.add_node("balanced", balanced_node)
builder.add_node("summary", nutrition_summary)
builder.add_node("track_meals", track_meals)
builder.add_node("track_remaining", track_remaining)
builder.add_node("exercise", suggest_exercise)

builder.add_edge("get_user_input", "classify")
builder.add_conditional_edges("classify", diet_router, {"loss": "loss", "gain": "gain", "balanced": "balanced"})
builder.add_edge("loss", "summary")
builder.add_edge("gain", "summary")
builder.add_edge("balanced", "summary")
builder.add_edge("summary", "track_meals")
builder.add_edge("track_meals", "track_remaining")
builder.add_edge("track_remaining", "exercise")
builder.add_edge("exercise", END)

In [ ]:
# STEP 12: Run

final_state = builder.compile().invoke({})

Namaste 🙏! I am NutriAI, your Indian dietician. Let's begin your health journey!
👤 What's your name? mrunal
🎂 Your age: 21
⚧️ Your gender (male/female/other): female
🏁 Your health goal? (weight loss/gain, healthy diet): weight gain
🌿 Diet preference (veg/non-veg/vegan): veg
🍛 Preferred Indian cuisine region: maharashtra
🩺 Any health issues? (PCOS, acne, thyroid, etc.): no
📏 Height in cm: 150
⚖️ Weight in kg: 36
🏃 Activity level (low, moderate, high): low
🎯 Goal classified as: Weight Gain

🍽️ Your Personalized Diet Plan:

Namaste NutriAI!  It's a pleasure to work with you.  Here's a sample Maharashtran vegetarian meal plan for weight gain, keeping in mind a 21-year-old female with no health issues.  Remember, this is a *sample* plan, and individual calorie needs vary.  It's best to consult with a doctor or registered dietitian for personalized recommendations.  This plan focuses on calorie-dense, nutritious foods common in Maharashtra.

**Important Note:** Portion sizes should be adjust